In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
from multiprocessing import shared_memory

flagshm= shared_memory.SharedMemory(name= 'flagname')
flag = np.ndarray((1,), dtype=np.int64, buffer=flagshm)
flag#다시 0으로 만들곳 어딘지 모르겠음
frameshm= shared_memory.SharedMemory(name= 'flagname')
flag = np.ndarray((,), dtype=, buffer=frameshm)

def getIntersectionArea(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    return (xB - xA + 1) * (yB - yA + 1)

cap = cv2.VideoCapture(0) # 삭제해도 됨
coord_list = []

person_model = YOLO("./weights/person.pt")
door_model = YOLO("./weights/door.pt")

while True:
    ret, frame = cap.read() # 여기서 프레임을 받아옴
    if ret:
        frame = cv2.flip(frame, 1) # 삭제해도 됨
        coord_list = []
        for coord in person_model.predict(frame)[0].boxes:
            x, y, w, h = coord.xywh[0][:]
            xmin, ymin, xmax, ymax = int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
            cv2.rectangle(frame, (xmin, ymin, xmax, ymax), (0, 0, 255), 4) # 삭제해도 됨
            coord_list.append([xmin, ymin, xmax, ymax])
            break
            
        for coord in door_model.predict(frame)[0].boxes:
            x, y, w, h = coord.xywh[0][:]
            xmin, ymin, xmax, ymax = int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 4) # 삭제해도 됨
            coord_list.append([xmin, ymin, xmax, ymax])
        
        distance = 0
        for i in range(len(coord_list) - 1): 
            for j in range(i+1, len(coord_list)):
                area = getIntersectionArea(coord_list[i], coord_list[j])
            if area > 0:
                # 여기서 신호 보내면 됨
                flag=???
                cv2.putText(frame, "WARNING!!", (50, 100), 1, 3, (0, 0, 255), 5, cv2.LINE_AA) # 삭제해도 됨
                break

        cv2.imshow("frame", frame) # 삭제해도 됨
    
        if cv2.waitKey(10) == 27: # 삭제해도 됨
            break # 삭제해도 됨
    
cap.release() # 삭제해도 됨
cv2.destroyAllWindows() # 삭제해도 됨


0: 480x640 1 Person, 20.9ms
Speed: 4.0ms preprocess, 20.9ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 21.7ms
Speed: 3.0ms preprocess, 21.7ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 19.0ms
Speed: 2.4ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 20.6ms
Speed: 2.0ms preprocess, 20.6ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 20.8ms
Speed: 1.5ms preprocess, 20.8ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 47.0ms
Speed: 4.3ms preprocess, 47.0ms inference, 6.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 53.0ms
Speed: 6.5ms preprocess, 53.0ms inference, 3.9ms postprocess 

Speed: 2.0ms preprocess, 19.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 18.6ms
Speed: 1.5ms preprocess, 18.6ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Container, 18.0ms
Speed: 2.0ms preprocess, 18.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 23.8ms
Speed: 2.6ms preprocess, 23.8ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Container, 24.1ms
Speed: 3.5ms preprocess, 24.1ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 20.3ms
Speed: 4.0ms preprocess, 20.3ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 16.9ms
Speed: 2.1ms preprocess, 16.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 30.0ms
Speed: 3.4ms preprocess, 30.0ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x


0: 480x640 (no detections), 49.8ms
Speed: 5.5ms preprocess, 49.8ms inference, 6.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 53.5ms
Speed: 5.4ms preprocess, 53.5ms inference, 5.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 19.4ms
Speed: 3.0ms preprocess, 19.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 22.2ms
Speed: 2.0ms preprocess, 22.2ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 19.5ms
Speed: 2.0ms preprocess, 19.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 39.5ms
Speed: 4.0ms preprocess, 39.5ms inference, 9.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 41.3ms
Speed: 5.0ms preprocess, 41.3ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 45.3ms
Speed: 5.0ms preprocess, 45.3ms inference, 6.0ms postprocess 

Speed: 3.2ms preprocess, 23.1ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 18.2ms
Speed: 2.9ms preprocess, 18.2ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 16.7ms
Speed: 2.8ms preprocess, 16.7ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 17.2ms
Speed: 2.7ms preprocess, 17.2ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 16.7ms
Speed: 2.4ms preprocess, 16.7ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 15.8ms
Speed: 3.1ms preprocess, 15.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Persons, 58.0ms
Speed: 6.0ms preprocess, 58.0ms inference, 8.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 55.2ms
Speed: 7.1ms preprocess, 55.2ms inference, 4.9ms postprocess per image at shape (1, 3, 48


0: 480x640 1 Person, 19.6ms
Speed: 1.0ms preprocess, 19.6ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 19.2ms
Speed: 2.0ms preprocess, 19.2ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 17.0ms
Speed: 3.7ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 16.4ms
Speed: 2.0ms preprocess, 16.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 45.3ms
Speed: 4.0ms preprocess, 45.3ms inference, 8.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 43.5ms
Speed: 3.8ms preprocess, 43.5ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 37.6ms
Speed: 4.0ms preprocess, 37.6ms inference, 7.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 35.2ms
Speed: 5.4ms preprocess, 35.2ms inference, 2.0ms postprocess 

Speed: 3.0ms preprocess, 23.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 35.9ms
Speed: 4.4ms preprocess, 35.9ms inference, 8.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 47.2ms
Speed: 6.4ms preprocess, 47.2ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 46.3ms
Speed: 5.8ms preprocess, 46.3ms inference, 10.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 53.2ms
Speed: 6.2ms preprocess, 53.2ms inference, 4.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 54.7ms
Speed: 5.2ms preprocess, 54.7ms inference, 8.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 52.5ms
Speed: 5.6ms preprocess, 52.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 51.5ms
Speed: 6.3ms preprocess, 51.5ms inference, 13.2ms postprocess per image at shape (1, 3, 480, 640


0: 480x640 1 Container, 47.2ms
Speed: 5.5ms preprocess, 47.2ms inference, 7.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 46.3ms
Speed: 6.5ms preprocess, 46.3ms inference, 7.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Container, 45.4ms
Speed: 6.8ms preprocess, 45.4ms inference, 9.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 56.1ms
Speed: 6.1ms preprocess, 56.1ms inference, 15.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Container, 52.3ms
Speed: 5.7ms preprocess, 52.3ms inference, 6.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Persons, 52.4ms
Speed: 5.4ms preprocess, 52.4ms inference, 10.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Container, 43.2ms
Speed: 5.3ms preprocess, 43.2ms inference, 4.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Person, 24.6ms
Speed: 2.0ms preprocess, 24.6ms inference, 3.9ms postprocess per image at 

In [ ]:
cap.release()
cv2.destroyAllWindows()